In [91]:
from pathlib import Path
import sys

dir_prj = Path('./').absolute().parent.parent
dir_src = dir_prj.parent/'geosaurus'/'src'
dir_raw = dir_prj/'data'/'raw'
sys.path.insert(0, str(dir_src))

from arcgis.gis import GIS
from arcgis import modeling
import dm

In [2]:
cntry_dm = dm.Country('USA')

cntry_dm

<dm.Country - USA (local)>

In [3]:
cntry_mdl = modeling.Country('US')

cntry_mdl

<modeling.Country - USA (local 2020)>

In [6]:
cntry_mdl = modeling.Country('USA', year=2019)

cntry_mdl

<modeling.Country - USA (local 2019)>

In [10]:
gis = GIS(username='knu2xs')

gis

Enter password:  ········


GIS @ https://knu2xs.maps.arcgis.com version:8.4

In [8]:
gis.url

'https://www.arcgis.com'

In [12]:
gis.users.me.username

'knu2xs'

In [13]:
import ast

In [29]:
module = dm

# __file__ returns string path to the __init__.py file, so the parent is the package directory
dir_pkg = Path(getattr(module, '__file__')).parent

dir_pkg

WindowsPath('d:/projects/demographic-modeling-module/src/dm')

In [31]:
pkg_fl_gen = dir_pkg.glob('**/*.py')

pkg_fl_gen = list(pkg_fl_gen)

pkg_fl_gen

[WindowsPath('d:/projects/demographic-modeling-module/src/dm/businesses.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/country.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/proximity.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/spatial.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/utils.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/_registry.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/_xml_interrogation.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/__init__.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/_accessor/_modify_geoaccessor.py'),
 WindowsPath('d:/projects/demographic-modeling-module/src/dm/_accessor/__init__.py')]

In [89]:
from io import TextIOWrapper
from typing import Union

import pandas as pd


def get_node_docstrings(nodes: list) -> list:
    ds_lst = [(i.name, ast.get_docstring(i)) for i in nodes]
    ds_lst = [i for i in ds_lst if i[1] is not None]
    return ds_lst


def get_function_docstrings(ast_node: Union[ast.ClassDef, ast.Module]) -> list:
    node_lst = [n for n in ast_node.body if isinstance(n, ast.FunctionDef)]
    ds_lst = get_node_docstrings(node_lst)
    return ds_lst


def get_class_docstrings(ast_module: ast.Module)->list:
    node_lst = [n for n in ast_module.body if isinstance(n, ast.ClassDef)]
    ds_lst = get_node_docstrings(node_lst)
    return ds_lst


def get_module_docstrings(module_path: Path)->list:
    
    # crack open the file and get the contents into an ast.Module instance
    with open(cntry_pth, 'r') as fb:
        file_contents = fb.read()
        module = ast.parse(file_contents)
    
    # get a list of all classes
    cls_lst = [n for n in module.body if isinstance(n, ast.ClassDef)]
    
    # get all class docstrings
    ds_lst = get_node_docstrings(cls_lst)
    
    # get all top level module docstrings
    ds_lst = ds_lst + get_function_docstrings(module)
    
    # iterate the classes in the module and get the child function docstrings
    for cls_node in cls_lst:
        ds_lst = ds_lst + get_function_docstrings(cls_node)
        
    return ds_lst

def get_package_docstrings(package_path:Path)->pd.DataFrame:
    mod_pth_gen = dir_pkg.glob('**/*.py')
    pkg_docs_lst = []
    for mod_pth in mod_pth_gen:
        mod_nm = mod_pth.stem
        if mod_nm != '__init__':
            mod_doc_lst = get_module_docstrings(mod_pth)
            mod_doc_lst = [(mod_nm, m[0], m[1]) for m in mod_doc_lst]
            pkg_docs_lst.extend(mod_doc_lst)
    df = pd.DataFrame(pkg_docs_lst, columns=['module_name', 'object_name', 'docstring'])
    return df

In [92]:
ds_df = get_package_docstrings(dir_pkg)

In [93]:
ds_df.head()

,module_name,object_name,docstring
0,businesses,Country,Country objects are instantiated by providing ...
1,businesses,DemographicModeling,DemograpphicModeling is a Pandas DataFrame acc...
2,businesses,_set_arcpy_ba_country,Helper function to set the country in ArcPy.
3,businesses,business,Access to business object instance for the cou...
4,businesses,enrich_variables,DataFrame of all the available geoenrichment v...


In [95]:
ds_df.docstring.to_json(dir_raw/'docstring.json')

In [154]:
with open(dir_raw/'docstring.json', 'w') as out_fl:
    for r in ds_df.docstring.iteritems():
        docstr = r[1]
        docstr = docstr.replace('\n', '\\n').replace('\t', '\\t').replace('"', '\\"')
        out_fl.write(f'{{"text": "{docstr}"}}\n')

In [153]:
docstr

"Create a closest destination dataframe using a destination Spatially Enabled\\nDataframe relative to the parent Spatially enabled DataFrame, but keep each\\norigin and destination still in a discrete row instead of collapsing to a\\nsingle row per origin. The main reason to use this is if needing the geometry\\nfor visualization.\\n\\nArgs:\\n    destination_dataframe:\\n        Destination points in one of the supported input formats.\\n    source:\\n        Optional - Either the path to the network dataset, the Country object\\n        associated with the Business Analyst source being used, or a GIS object\\n        instance. If invoked from a dataframe created for a country's standard\\n        geography levels using the dm accessor, get_nearest will use the parent\\n        country properties to ascertain how to perform the networks solve.\\n    single_row_per_origin:\\n        Optional - Whether or not to pivot the results to return\\n        only one row for each origin location